## Step 1: link to the API

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
import requests
from config import key
from config import gkey

In [2]:
#get state code number --CA 06
state_url = "https://aqs.epa.gov/data/api/list/states?email="+gkey+"&key="+key
response = requests.get(state_url)
response_json = response.json()
response_json_data = response_json['Data']
#print(response_json_data)
state_df = pd.DataFrame.from_dict(response_json_data)
state_df.head()

,code,value_represented
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


In [3]:
#get SF code -- 075
city_url = "https://aqs.epa.gov/data/api/list/countiesByState?email="+gkey +"&key="+key+"&state=06"
response = requests.get(city_url)
response_json = response.json()
response_json_data = response_json['Data']
#print(response_json_data)
city_df = pd.DataFrame.from_dict(response_json_data)
city_df.head()

,code,value_represented
0,001,Alameda
1,003,Alpine
2,005,Amador
3,007,Butte
4,009,Calaveras


In [ ]:
#CBSAs code": "41860",
#      "value_represented": "San Francisco-Oakland-Hayward, CA

#  "Parameter code": "PM2.5 CONT NONREF",
#       "value_represented": "PM2.5 Continuous, Nonreference Methods"
#     },
#     {
#       "code": "PM2.5 MASS/QA",
#       "value_represented": "PM2.5 Mass and QA Parameters"


# "CRITERIA code": "88101",
#       "value_represented": "PM2.5 - Local Conditions"

In [ ]:
# state_codes = state_df['code'].tolist()
# state_codes_test = [state_codes[0], state_codes[1], state_codes[2]]
# total = []

In [ ]:
# for i in state_codes_test:
#     state_data = 'https://aqs.epa.gov/data/api/sampleData/byState?email='+gkey+'&key='+key+'&param=88101&bdate=20140301&edate=20190331&state='+i
#     response = requests.get(state_data)
#     try:
#         response_json = response.json()
#         response_json_data = response_json['Data']
#         total.append(pd.DataFrame.from_dict(response_json_data))
#     except:
#         continue

In [ ]:
ca_url = "https://aqs.epa.gov/data/api/dailyData/byState?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06"

response = requests.get(ca_url)
response_json = response.json()
#print(response_json)
response_json_data = response_json['Data']
# #print(response_json_data)
ca_df = pd.DataFrame.from_dict(response_json_data)
ca_df.head()


In [ ]:
ca_5year = total.loc[total['pollutant_standard']=="PM25 24-hour 2012"]

In [ ]:
#get current 1y PM 2.5 data for SF
#https://aqs.epa.gov/data/api/sampleData/byCBSA?email=test@aqs.api&key=test&param=42602&bdate=20170101&edate=20170101&cbsa=16740
#sf_url = "https://aqs.epa.gov/data/api/sampleData/byCBSA?email="+gkey+"&key="+key+"&param=88101&bdate=20190101&edate=20190101&cbsa=41860"

sf_url = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county=075"

response = requests.get(sf_url)
response_json = response.json()
#print(response_json)
response_json_data = response_json['Data']
# #print(response_json_data)
sf_df = pd.DataFrame.from_dict(response_json_data)
sf_df.head()

#https://aqs.epa.gov/data/api/monitors/byState?email=test@aqs.api&key=test&param=42401&bdate=20150501&edate=20150502&state=15

In [ ]:
#choose only "pm2.5 24h standard"
sf_df_new = sf_df.loc[sf_df['pollutant_standard']=="PM25 24-hour 2012"]
sf_df_new

In [ ]:
#get min,max,mean
aqi_min = sf_df_new['aqi'].min()
aqi_max =sf_df_new['aqi'].max()
aqi_mean = sf_df_new['aqi'].mean()

print(aqi_min)
print(aqi_max)
print(aqi_mean)

In [ ]:
# summurize data in categories
size_bins = [0,50,100,150,200,300,500]
size_names= ['good','Moderate','Unhealthy for sensitive groups','unhealthy','very unhealthy','hazardous']
sf_df_bin = sf_df_new[['date_local']].groupby(pd.cut(sf_df_new['aqi'],bins=size_bins,labels=size_names)).count()
sf_df_bin

## Step 2: export data to the database

In [ ]:
sf_df_new.to_csv("sf_aqi",index=False)

In [ ]:
ca_5year.to_csv('national_data', index = False)